Here's my repo link: https://github.com/CaptainMaxine/A2A-MCP

# Multi-Agent Customer Service System (A2A + MCP)

This notebook demonstrates an end-to-end multi-agent customer service
system with:

- RouterAgent (orchestrator, LLM-backed intent classification)
- CustomerDataAgent (MCP-backed DB specialist)
- SupportAgent (LLM-backed response generator)
- A2ACoordinator (logs A2A steps and final answer)

We run all required scenarios:
- Simple query
- Coordinated upgrade
- Complex active-customers-with-open-tickets
- Escalation (double charge refund)
- Multi-intent (update email + history)
- Scenario 1/2/3 from the assignment spec.


In [1]:
!git clone https://github.com/CaptainMaxine/A2A-MCP.git
%cd A2A-MCP

Cloning into 'A2A-MCP'...
remote: Enumerating objects: 166, done.
remote: Counting objects: 100% (166/166), done.
remote: Compressing objects: 100% (155/155), done.
remote: Total 166 (delta 69), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (166/166), 75.93 KiB | 3.80 MiB/s, done.
Resolving deltas: 100% (69/69), done.
/content/A2A-MCP


In [2]:
!pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.0/334.0 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.7/125.7 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.0/95.0 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 40.3 MB/s eta 0:00:00
  Attempting uninstall: pytest
    Found existing installation: pytest 8.4.2
    Uninstalling pytest-8.4.2:
      Successfully uninstalled pytest-8.4.2


In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "sk-XXXX"

In [4]:
from mcp_server.server import ensure_database
ensure_database()   # creates tables + sample data

[MCP Server] Creating database at /content/A2A-MCP/mcp_server/support.db ...
Connected to database: /content/A2A-MCP/mcp_server/support.db
Tables created successfully!
Triggers created successfully!
Sample data inserted successfully!
  - 15 customers added
  - 25 tickets added
Database connection closed.
[MCP Server] Database initialized with sample data.
[MCP Server] Database stored at: /content/A2A-MCP/mcp_server/support.db


In [5]:
from agents.coordinator import A2ACoordinator
coord = A2ACoordinator()

In [6]:
def run_scenario(query: str, coord: A2ACoordinator):
    print("\n" + "="*80)
    print(f"USER QUERY: {query}")
    print("="*80)

    response = coord.run(query)
    print("\n--- FINAL RESPONSE ---")
    print(response)


In [7]:
queries = [
    # 1. Simple Query
    "Get customer information for ID 5",

    # 2. Coordinated Query
    "I'm customer 1 and need help upgrading my account",

    # 3. Complex Query
    "Show me all active customers who have open tickets",

    # 4. Escalation
    "I'm customer 1 and I've been charged twice, please refund immediately!",

    # 5. Multi-Intent
    "I'm customer 1, update my email to new@email.com and show my ticket history",
]

for q in queries:
    run_scenario(q, coord)



USER QUERY: Get customer information for ID 5
[CustomerDataAgent] Received: scenario=simple_get, content=Get customer information for ID 5

--- FINAL RESPONSE ---
("Subject: Customer Information for ID 5\n\nDear [User's Name],\n\nThank you for reaching out. I’m happy to assist you with the information you need for customer ID 5.\n\nHere are the details for Charlie Brown:\n- Status: Active\n- Email: charlie.brown@email.com\n\nIf you require any further assistance or have additional questions, please feel free to ask.\n\nBest regards,\n\n[Your Name]  \n[Your Position]  \n[Your Company]  ", ['[STEP 1] user → router | content=Get customer information for ID 5 | state={}', "[STEP 2] router → customer_data | content=Get customer information for ID 5 | state={'intents': ['simple_get'], 'customer_id': '5', 'scenario': 'simple_get', 'original_query': 'Get customer information for ID 5'}", "[STEP 3] customer_data → router | content=customer_context_ready | state={'intents': ['simple_get'], 'cus

## Conclusion

In this assignment, I implemented a multi-agent customer service system
with three specialized agents: a RouterAgent for LLM-based intent
classification and task allocation, a CustomerDataAgent as a deterministic
MCP-backed data specialist, and a SupportAgent that uses an LLM to generate
polished, user-facing responses. The agents coordinate via an A2A-style
message loop managed by the A2ACoordinator, which logs each step of the
interaction (router → data → support → router → user).

The main challenges were making the coordination robust across multiple
scenarios (task allocation, negotiation, and multi-step queries) and
ensuring that information was not lost when state moved between agents.
I addressed this by defining a shared A2AMessage structure, using a single
`state` dictionary to carry context, and adding explicit, human-readable
logging at every transfer. Another challenge was combining deterministic
database logic with model-driven behavior. I solved this by keeping all
data access and updates in the CustomerDataAgent, while using LLMs only
for intent classification and natural language response generation, which
keeps the system both interpretable and flexible.
